# Distributed Databases and Big Data

# Solution Workbook



**About Jupyter Notebook**

*Server Information:*

    The version of the notebook server is 4.2.3 and is running on:
        Python 3.5.2 


*Current Kernel Information:*
    
    Python 3.5.2 
    IPython 5.1.0 

*Kindly run .ipynb file from the folder extracted from zip inplace, or by opening the file in jupyter by directing it to the mentioned folder.*

# Table of Contents

- [Parallel Sort](#3)
    

## Libraries

***`If the library isn't installed already, please unhash and run the code below.`***


In [1]:
#!pip install pandas
#!pip install multiprocessing

In [3]:
import pandas as pd
from multiprocessing import Pool

## data collection

In [4]:
# CLIMATE DATA

#read excel into dataframe
cdf = pd.read_csv('ClimateData.csv')

#create list
cd = []
row = []
for i in range(cdf.shape[0]):
    for j in range(cdf.shape[1]):
        row.append(cdf.iloc[i,j])
    cd.append(row)
    row = []

In [5]:
# FIRE DATA 

#read excel into dataframe
fdf = pd.read_csv('FireData.csv')

#creating list
fd = []
row = []
for i in range(fdf.shape[0]):
    for j in range(fdf.shape[1]):
        row.append(fdf.iloc[i,j])
    fd.append(row)
    row = []


***

# Parallel Sort

<a id='3'></a>

## Parallel Sort 

1. Write an algorithm to sort ​ fire data ​ based on ​ surface temperature ​ ​ (°C) ​ in a ​ ascending
order. ​ Justify your choice of the data partition technique and sorting technique you have
used.

**Solution:**

* Sorting:

**Parallel Redistribution Merge All Sort**. This is because among the 5 different sorting algorithems the redistribution versions of binary and merge all are more efficient compared to the non redistriubtion ones as through redistribution true parralelism is achieved rather than using a few processors for cetain stages. 

Between the two redistribution methods Merge all is more efficient compared to binary as it results in a smaller tree compared to binary and also with larger datasets binary can have a huge over head due to pipelining. 

Partition sort would have been the most efficient if the problem of load ballancing would have been solved. Hence in this case Parallel Merge All sort is used to conduct the sorting

* Partitioning:

for the initial data placement into the processors **Round Robin** is used. This is so that the processors start with an equal workload when doing the sorting phase.

For the redistribution phase **Range Partitioning** is used as by the Parralel Redistribution Merge All method. This is so that each processor gets the data in the correct order so that once the k way merging is complete by each processor. The union of the data in the processors in order provides the sorted dataset. 

### Data Pratitioning Algorithems to be used

##### Round Robin: for original data placement to proessors

In [48]:
# Round-robin data partitionining function
def rr_partition(data, n):
    """
    Perform data partitioning on data
    Arguments:
    data -- an input dataset which is a list
    n -- the number of processors
    Return:
    result -- the paritioned subsets of D
    """
    
    result = []
    
    for i in range(n):
        result.append([])
    
    ### START CODE HERE ###
    
    # For each bin, perform the following
    for index, element in enumerate(data):
        # Calculate the index of the bin that the current data point will be assigned
        index_bin = (int) (index % n)
        #print(str(index) + ":" + str(element))
        result[index_bin].append(element)
    
    ### END CODE HERE ###
    return result

##### Range Partition: To perfrom redistrution phase for k way merging within processes after each processor has done local sort.

In [66]:
#first we use Round Robin Method to redistribute the data into 4 processors. 
# Range data partitionining function
def range_partition(data, range_indices):
    
    """
    Perform range data partitioning on data
    Arguments:
    data -- an input dataset which is a list
    range_indices -- the index list of ranges to be split 
    Return:
    result -- the paritioned subsets of D
    """
    
    result = []
    
    
    # First, we sort the dataset according their values
    new_data = sorted(data, key= lambda x: x[7])
    
    
    # Calculate the number of bins - 1
    n_bin = len(range_indices)
    
    
    # For each bin, perform the following
    for i in range(n_bin):
        
        # Find elements to be belonging to each range
        s = [x for x in new_data if x[7] < range_indices[i]]
        
        # Add the partitioned list to the result
        result.append(s)
        
        if len(s) > 0:
            # Find the last element in the previous partition
            last_element = s[len(s)-1]

            # Find the index of of the last element
            last = new_data.index(last_element)

            # Remove the partitioned list from the dataset
            new_data = new_data[int(last)+1:]
    
    # Append the last remaining data list
    result.append([x for x in new_data if x[7] >= range_indices[n_bin-1]])
    
    
    return result

#### in order to try and minimize skew we will use data quantiles to create range for partitioning. 
We use the first, second and third quantiles.

In [67]:
#identify best partitioning range to gain almost equal partitions
fdf['Surface Temperature (Celcius)'].quantile([0.25, 0.5, 0.75])

0.25    44.0
0.50    51.0
0.75    61.0
Name: Surface Temperature (Celcius), dtype: float64

### Sorting Algorthims to be used

#### this is for phase 1 before redistribution. each processor will sort their data locally.
The serial external sorting will require several functions. The functions it will use are as follows:

**sorting phase:** Sort subsets created according to buffer size
    * qsort() : this is to sort the subsets created out of the original data. These subsets are small enough to fit in the main memory.

**merging phase:** merging the sorted subsets, chunks at a time so they can fit in the buffer size. 1 buffer is used for output while the rest are for the subsets to merge. 
    * find_min() : this is used to identify the min value among the merging subsets.
    * k_way_merge() : this the actual algorithem that with the help of find_min() merges the subsets in a sorted order.

##### local q-sort

In [68]:
def qsort(arr):

    """
    Quicksort a list of lists
    Arguments:
    arr -- the input list of list to be sorted
    Return:
    result -- the sorted arr
    """

    if len(arr) <= 1:
        return arr
    else:
        return qsort([x for x in arr[1:] if x[7] < arr[0][7]]) \
            + [arr[0]] \
            + qsort([x for x in arr[1:] if x[7] >= arr[0][7]])

In [69]:
#test qsort
#qsort(fd[0:3])

##### find_min and k_way_merge Algorithems

In [70]:
# Let's first look at 'k-way merging algorithm' that will be used
# to merge sub-record sets in our external sorting algorithm.
import sys


# Find the smallest record
def find_min(records):
    
    """
    Find the smallest record
    Arguments:
    records -- the input record set
    Return:
    result -- the smallest record's index
    """
    
    m = records[0][-1] #select the temp value of the first row and store it as the minimum
    index = 0
    
    for i in range(len(records)):
        if(records[i][-1] < m): #if we find a value smaller than our initial minimum, replace the original with the current.
            index = i
            m = records[i][-1]
    
    return index #resurns the index at which the row with the miminum temp is at. 



def k_way_merge(record_sets):
    
    """
    K-way merging algorithm
    Arguments:
    record_sets -- the set of mulitple sorted sub-record sets
    Return:
    result -- the sorted and merged record set
    """
    
    # indexes will keep the indexes of sorted records in the given buffers
    indexes = []
    
    for x in record_sets:
        indexes.append(0) # initialisation with 0
    
    
    # final result will be stored in this variable
    result = []
    
    # the merging unit (i.e. # of the given buffers)
    tuple = []
    
    while(True):
        tuple = [] # initialise tuple
        
        # This loop gets the current position of every buffer
        for i in range(len(record_sets)):
            
            if(indexes[i] >= len(record_sets[i])):
                tuple.append([0,0,0,0,0,0,0,sys.maxsize]) #if the record set has been emptied, just add a max temp record.
            else:
                tuple.append(record_sets[i][indexes[i]]) #if not empty add the next value to tuple for identifying minimum
        
        # find the smallest record and identify which record set it belongs in
        smallest = find_min(tuple)
        
        # if we only have sys.maxsize on the tuple, we reached the end of every record set
        if(tuple[smallest] == [0,0,0,0,0,0,0,sys.maxsize]):
            break
        
        # This record is the next on the merged list
        result.append(record_sets[smallest][indexes[smallest]])
        indexes[smallest] +=1
        
    return result

In [71]:
# Testing k_way_merge()
# a = fd[0:3]
# b = fd[3:6]
# c = fd[6:9]
# l = [a,b,c]
# k_way_merge(l)

#### The final serial external sorting algorithem to be used.

In [72]:
def serial_sorting(dataset, buffer_size):

    """
    Perform a serial external sorting method based on sort-merge
    The buffer size determines the size of eac sub-record set
    Arguments:
    dataset -- the entire record set to be sorted
    buffer_size -- the buffer size determining the size of each sub-record
    set
    Return:
    result -- the sorted record set
    """
    
    if (buffer_size <= 2):
        print("Error: buffer size should be greater than 2")
        return

    result = []
    
    # --- Sort Phase ---
    sorted_set = []
    
    # Read buffer_size pages at a time into memory and
    # sort them, and write out a sub-record set (i.e. variable: subset)
    start_pos = 0
    N = len(dataset)
    while True:
        if ((N - start_pos) > buffer_size):
            # read B-records from the input, where B = buffer_size
            subset = dataset[start_pos:start_pos + buffer_size]
            # sort the subset (using qucksort defined above)
            sorted_subset = qsort(subset)

            sorted_set.append(sorted_subset)
            start_pos += buffer_size
        else:
            # read the last B-records from the input, where B is less than buffer_size
            subset = dataset[start_pos:]
            # sort the subset (using qucksort defined above)
            sorted_subset = qsort(subset)
            sorted_set.append(sorted_subset)
            break
    
    # --- Merge Phase ---
    merge_buffer_size = buffer_size - 1
    dataset = sorted_set
    while True:
        merged_set = []

        N = len(dataset)
        start_pos = 0
        
        
        while True:
            
            if ((N - start_pos) > merge_buffer_size):
                # read C-record sets from the merged record sets, where C = merge_buffer_size
                subset = dataset[start_pos:start_pos + merge_buffer_size]
                merged_set.append(k_way_merge(subset)) # merge lists in subset
                start_pos += merge_buffer_size
            else:
                # read C-record sets from the merged sets, where C is less than merge_buffer_size
                subset = dataset[start_pos:]
                merged_set.append(k_way_merge(subset)) # merge lists in subset
                break
                
        dataset = merged_set
        
        if (len(dataset) <= 1): # if the size of merged record set is 1, then stop
            result = merged_set[0] #its a list within a list so just get the inner list
            break
            
    return result

In [73]:
# Testing serial_sorting
# serial_sorting(fd[0:3], 4)

### Parallel Redistribution Merge All Sort

Using all the above algorithems Parallel Redistribution Merge All Sort sorts the dataset efficiently.

In [83]:
# Parallel searching algorithm for range selection
# range partition and binary search

def parallel_red_ma_sort(data, n_processor, partition_range):
    
    """
    Perform parallel redistribution merge all sort for the given data between a given number of processors. Range distribution \
    is used and then k way merge to finally sort the data. 

    Arguments:
    data -- the input dataset which is a list of lists
    n_processor -- the number of parallel processors
    partition_rang -- the range to be used to create bins for range partition
    
    Return:
    results -- sorted data in each processor
    """
    
    processes = [] #to store worker process objects
    
    pool = Pool(processes = n_processor+1) #1 kept for for loops
    
    #first distribute data equally in all processors using round robin data partition
    part_data = rr_partition(data, n_processor)
    
    
    
    #local sort is performed in each processor
    for subset in part_data:
        process = pool.apply_async(serial_sorting, args=(subset, n_processor,))
        processes.append(process)
    
    #result after local sort is done and partitioned data is now in sorted form
    sorted_part_data = [p.get() for p in processes]
    processes = [] #empty processes        
    
    #each processors data set is range partitioned for each processor
    for subset in sorted_part_data:
        process = pool.apply_async(range_partition, args=(subset, partition_range))
        processes.append(process)
    
    #store output from processors in ranged partitioned data for each processor to merge
    ranged_sorted_part_data = []
    for i in range(len(partition_range)+1):
        ranged_sorted_part_data.append([])
    
    #create datasets for each processor to k way merge
    
    for p in processes:
        p_output = p.get()
        for i in range(len(partition_range)+1):
            ranged_sorted_part_data[i].append(p_output[i])
    
    #empty processes
    processes = []
    
    #each processor does a k way merge on its ranged and sorted data sets
    for subset in ranged_sorted_part_data:
        process = pool.apply_async(k_way_merge, args=(subset,))
        processes.append(process)
        
    #each processor holds it own sorted set; union of all sets in order is the sorted dataset. 
    final_sort_data_for_process = [p.get() for p in processes]
    
    #close pool
    pool.close()
    
    #kmerge all data from proceses to obtain sorted data set
    result = k_way_merge(final_sort_data_for_process)

#    to see sorted only for temperature; easy to check
#     for row in result:
#         print(row[7])
    
    return result
    
    

### Solution

In [87]:
# Calling the function using 4 processors and providing the ranges partition by using quantiles as checked above
parallel_red_ma_sort(fd, 4, [44, 51, 61])

[[-37.885999999999996,
  147.20699999999999,
  302.0,
  '2017-07-02T04:28:42',
  10.699999999999999,
  50,
  '2017-07-02',
  28],
 [-37.885999999999996,
  147.20699999999999,
  302.0,
  '2017-07-02T04:28:42',
  10.699999999999999,
  50,
  '2017-07-02',
  28],
 [-36.943000000000005,
  143.286,
  302.69999999999999,
  '2017-11-11T15:08:00',
  18.800000000000001,
  51,
  '2017-11-11',
  29],
 [-37.061999999999998,
  141.37299999999999,
  303.10000000000002,
  '2017-07-01T13:11:41',
  16.100000000000001,
  53,
  '2017-07-01',
  29],
 [-37.466000000000001,
  148.09999999999999,
  302.19999999999999,
  '2017-10-02T23:44:31',
  10.9,
  50,
  '2017-10-02',
  29],
 [-37.061999999999998,
  141.37299999999999,
  303.10000000000002,
  '2017-07-01T13:11:41',
  16.100000000000001,
  53,
  '2017-07-01',
  29],
 [-37.226999999999997,
  141.14600000000002,
  305.10000000000002,
  '2017-10-03T01:22:44',
  41.200000000000003,
  54,
  '2017-10-03',
  31],
 [-37.380000000000003,
  149.334,
  304.5,
  '2017


--------------------------------------------------------------------------------------------------------------------------